# Generation of random catalogs from simulation

The idea is to create several catalogs from a whole set of simulated stars in one CCD (Axel's simulations).

Each catalog should contain:
- 50 test stars (without noise, non-shifted and one at each CCD-pixel)
- 50 stars to fit the model (with noise, shifted and randomly chosen)

### Imports

In [27]:
import numpy as np
import scipy.stats as sst
import matplotlib.pyplot as plt
import progressbar as pb
import copy
import sep
import os

from astropy.io import fits
from shapepipe.pipeline import file_io

### User parameters

In [28]:
# Location of the file containing raw simulated stars
CCD_number = 38
simu_path = '/Users/mschmitz/jbonnin/Data/CFIS_PSF'

# Directory used to store generated catalogs
output_path = '/Users/mschmitz/jbonnin/Data/CFIS_PSF/CCD-{}_sexdataset'.format(CCD_number)

# Path of a model file for creating .fits catalogs
basic_fits = '/Users/mschmitz/jbonnin/Data/all_w3_star_cat/star_selection-2079613-0.fits'

sexmode = True

### Pre-defined functions

In [29]:
def create_fits(fits_path, data_dict, fits_ex = basic_fits):
    """
    Create a .fits file
    INPUTS:
        fits_path: path to the future .fits file
        data_dict: dictionnary where data is stored (usually {'VIGNET': ..., 'XWIN_IMAGE': ..., ...})
        fits_ex: path to an existing .fits file used as a model
    """
    fits_file = file_io.FITSCatalog(fits_path, open_mode = file_io.BaseCatalog.OpenMode.ReadWrite, SEx_catalog=True)
    fits_file.save_as_fits(data_dict, sex_cat_path = basic_fits)


def add_noise_std(images, sigma, seed = 0):
    N = images.shape[0]
    x_size = images.shape[1]
    y_size = images.shape[2]

    np.random.seed(seed)
    noise = np.array([sst.norm.rvs(scale = sigma, size = x_size * y_size) for k in range(N)])
    noisy_images = images + noise.reshape(N, x_size, y_size)

    return noisy_images


def run_sep(vign, detect_thresh=1.5):
    """
    """
    
    vign_tmp = vign.byteswap().newbyteorder()
    
    # Get background 
    bkg = sep.Background(vign_tmp)
    
    # Detection
    obj = sep.extract(vign_tmp, detect_thresh, err=bkg.globalrms)
    
    if len(obj) == 0:
        raise ValueError("No object detected")
    elif len(obj) > 1:
        raise ValueError("More than one object detected")
    
    # Flux estimation
    kronrad, krflag = sep.kron_radius(vign_tmp, obj['x'], obj['y'], obj['a'], obj['b'], obj['theta'], 6.0)
    flux, fluxerr, flag = sep.sum_ellipse(vign_tmp, obj['x'], obj['y'], obj['a'], obj['b'], obj['theta'], 2.5*kronrad,subpix=1,err=bkg.globalrms)
    r, flag = sep.flux_radius(vign_tmp, obj['x'], obj['y'], 6.*obj['a'], 0.5,normflux=flux, subpix=5)
    
    output = {'x': obj['x'][0],
              'y': obj['y'][0],
              'a': obj['a'][0],
              'b': obj['b'][0],
              'elongation': obj['a'][0]/obj['b'][0],
              'flux_auto': flux[0],
              'fluxerr_auto': fluxerr[0],
              'flux_radius': r[0]}
    
    return output

def sexparam(stars):
    n = stars.shape[0]
    sexparams = {'x': np.zeros((n)), 'y': np.zeros((n)), 'a': np.zeros((n)), 'b': np.zeros((n)),
                 'elongation': np.zeros((n)), 'flux_auto': np.zeros((n)), 'fluxerr_auto': np.zeros((n)),
                 'flux_radius': np.zeros((n))}
    print('Processing stars to recover SExtractor parameters ...')
    for k in pb.progressbar(range(n)):
        params = run_sep(stars[k].byteswap().newbyteorder())
        sexparams['x'][k] = params['x']
        sexparams['y'][k] = params['y']
        sexparams['a'][k] = params['a']
        sexparams['b'][k] = params['b']
        sexparams['elongation'][k] = params['elongation']
        sexparams['flux_auto'][k] = params['flux_auto']
        sexparams['fluxerr_auto'][k] = params['fluxerr_auto']
        sexparams['flux_radius'][k] = params['flux_radius']
        
    print('Done')
    
    return sexparams

### Load the data

In [30]:
simu = np.load('{}/psf_ccd_{}.npy'.format(simu_path, CCD_number), allow_pickle = True).item()
print(simu.keys())

N_stars = len(simu['CCD_n'])
print('{} stars in CCD {}'.format(N_stars, CCD_number))

dict_keys(['vignet', 'e1_true', 'e2_true', 'fwhm', 'CCD_n', 'X', 'Y'])
24823 stars in CCD 38


In [31]:
# Identifying CCD-pixels

X_px = np.sort(list(set(simu['X'])))
Y_px = np.sort(list(set(simu['Y'])))

x_gap = int(X_px[1] - X_px[0])
print('Width of a CCD_pixel: {}'.format(x_gap))

y_gap = int(Y_px[1] - Y_px[0])
print('Height of a CCD_pixel: {}'.format(y_gap))

Width of a CCD_pixel: 409
Height of a CCD_pixel: 461


In [32]:
# Recovering index of stars present in each CCD-pixel

px_pos = [[[] for y in Y_px] for x in X_px]

for k in range(N_stars):
    for i in range(len(X_px)):
        for j in range(len(Y_px)):
            if (simu['X'][k] == X_px[i]) and (simu['Y'][k] == Y_px[j]):
                px_pos[i][j].append(k)

N_stars_pCCD = np.array([[len(stars) for stars in px_col] for px_col in px_pos])
print(np.array(N_stars_pCCD).T)

min_px = np.min(N_stars_pCCD)
print('Minimum number of stars per CCD-pixel: {}'.format(min_px))

[[470 521 526 470 480]
 [526 489 529 517 480]
 [473 501 458 516 500]
 [505 509 457 521 491]
 [518 502 495 497 496]
 [462 489 484 495 523]
 [453 473 466 505 511]
 [526 517 469 483 528]
 [469 475 490 515 511]
 [542 505 477 514 494]]
Minimum number of stars per CCD-pixel: 453


### Select stars

In [33]:
N_catalogs = min_px // 2

flat_idx = [idx for col in px_pos for idx in col]
slices_idx = [list(s) for s in zip(*flat_idx)]

test_slices_idx = slices_idx[:N_catalogs]
train_slices_idx = slices_idx[N_catalogs:-1]

train_idx = np.array(train_slices_idx).flatten()

### Build test catalogs

In [42]:
test_dicts = [{'VIGNET': simu['vignet'][s], 'XWIN_IMAGE': simu['X'][s], 'YWIN_IMAGE': simu['Y'][s], 'NUMBER': np.arange(1, len(s)+1)} for s in slices_idx[:N_catalogs]]

if os.path.isdir(output_path):
    os.system('rm ' + output_path + '/test-star_selection*')
else:
    os.system('mkdir ' + output_path)
print('Building test stars catalogs ...')
for n in pb.progressbar(range(N_catalogs)):
    create_fits('{}/test-star_selection-{:07d}-{}.fits'.format(output_path, n, CCD_number), test_dicts[n])
print('Done')

  0% (1 of 226) |                        | Elapsed Time: 0:00:00 ETA:   0:00:36

Building test stars catalogs ...


100% (226 of 226) |######################| Elapsed Time: 0:00:07 Time:  0:00:07


Done


### Shift train stars

In [35]:
np.random.seed(0)
x_shifts = np.random.uniform(- x_gap / 2, x_gap / 2, len(train_idx))
y_shifts = np.random.uniform(- y_gap / 2, y_gap / 2, len(train_idx))

simu_s = copy.deepcopy(simu)

simu_s['X'][train_idx] = simu['X'][train_idx] + x_shifts
simu_s['Y'][train_idx] = simu['Y'][train_idx] + y_shifts

### Add noise

In [36]:
sig_noise = 1e-3

simu_sn = copy.deepcopy(simu_s)
simu_sn['vignet'][train_idx] = add_noise_std(simu['vignet'][train_idx], sig_noise)

### Select train stars

In [37]:
np.random.seed(0)
rand_train_idx = np.copy(train_idx)
np.random.shuffle(rand_train_idx)
train_selections = [rand_train_idx[(50 * n):(50 * (n + 1))] for n in range(N_catalogs)]

### Build train catalogs

In [38]:
if sexmode:
    sexparams = sexparam(simu_sn['vignet'])
    train_dicts = [{'VIGNET': simu_sn['vignet'][s].astype('float32'), 'XWIN_IMAGE': simu_sn['X'][s], 'YWIN_IMAGE': simu_sn['Y'][s],
                    'FLUX_RADIUS': sexparams['flux_radius'][s].astype('float32'), 'SNR_WIN': np.sqrt(np.sum(simu['vignet'][s] ** 2, axis = (1,2))) / sig_noise,
                    'FLUX_AUTO': sexparams['flux_auto'][s].astype('float32'), 'FLUXERR_AUTO': sexparams['fluxerr_auto'][s].astype('float32'),
                    'ELONGATION': sexparams['elongation'][s].astype('float32'), 'FLAGS': np.zeros((len(s)), dtype = 'int16'), 
                    'NUMBER': np.arange(1,len(s)+1, dtype = 'int32')} for s in train_selections]
else:
    train_dicts = [{'VIGNET': simu_sn['vignet'][s], 'XWIN_IMAGE': simu_sn['X'][s], 'YWIN_IMAGE': simu_sn['Y'][s]} for s in train_selections]

# 0.186    
    
if os.path.isdir(output_path):
    os.system('rm ' + output_path + '/train-star_selection*')
else:
    os.system('mkdir ' + output_path)
print('Building train stars catalogs ...')
for n in pb.progressbar(range(N_catalogs)):
    create_fits('{}/train-star_selection-{:07d}-{}.fits'.format(output_path, n, CCD_number), train_dicts[n])
print('Done')

  0% (98 of 24823) |                     | Elapsed Time: 0:00:00 ETA:   0:00:50

Processing stars to recover SExtractor parameters ...


100% (24823 of 24823) |##################| Elapsed Time: 0:00:43 Time:  0:00:43
N/A% (0 of 226) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Done
Building train stars catalogs ...


100% (226 of 226) |######################| Elapsed Time: 0:00:09 Time:  0:00:09


Done
